In [9]:
import os
from PIL import Image

# Directory path
folder_path = "/Users/tylerklimas/Desktop/FoodClassifier/data/pizza_steak_sushi/train/pizza"


image_paths = []
images = []
# Check if the directory exists
if os.path.exists(folder_path):
    # Get list of files in the directory
    files = os.listdir(folder_path)
    
    # Loop through each file
    for file in files:
        # Check if file is an image
        if file.endswith('.jpg') or file.endswith('.png'):  # Add more extensions if needed
            # Open the image
            image_path = os.path.join(folder_path, file)
            image_paths.append(image_path)
            image = Image.open(image_path)
            images.append(image)

else:
    print("Directory not found!")

In [22]:
from torch.utils.data import DataLoader
from torchvision import transforms



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
]
)
dataset = [(transform(image),) for image in images]



train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [23]:
from transformers import AutoImageProcessor, ViTForImageClassification
import torch
from datasets import load_dataset

# dataset = load_dataset("huggingface/cats-image")
# image_path = image_paths[1]



# dataset = load_dataset("huggingface/cats-image")
# cat_image = dataset["test"]["image"][0]
# print(cat_image)

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")


for data in train_loader:
    inputs = image_processor(data, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    # model predicts one of the 1000 ImageNet classes
    predicted_label = logits.argmax(-1).item()
    print(model.config.id2label[predicted_label])


ValueError: axes don't match array